In [0]:
dbutils.widgets.removeAll()

In [0]:
dbutils.widgets.text("source_catalog", "flightstats_historical_dev_azr_westus")
source_catalog = dbutils.widgets.get("source_catalog")

dbutils.widgets.text("source_schema", "silver")
source_schema = dbutils.widgets.get("source_schema")

dbutils.widgets.text("source_table", "flightstats_clean")
source_table = dbutils.widgets.get("source_table")

In [0]:
tables_config = {
    "flightstats_historical_dev_azr_westus": {
        "silver": {
            "flightstats_clean": {
                "target_catalog": "flightstats_historical_dev_azr_westus",
                "target_schema": "data_contract_dev",
                "target_table": "flightstats",
                "projected_cols": "departure_airport,departure_gate,economy_number_of_seats,aircraftmanufacturer,fs_operating_airline_code as airline,operating_flight_number as flight_number",
            }
        }
    }
}

In [0]:
catalog_config = tables_config.get(source_catalog)
schema_config = catalog_config.get(source_schema)
src_table_config = schema_config.get(source_table)
target_catalog = src_table_config.get("target_catalog")
target_schema = src_table_config.get("target_schema")
target_table = src_table_config.get("target_table")
projected_cols = src_table_config.get("projected_cols")

In [0]:
df = (
    spark.sql(
        f"""select {projected_cols} from {source_catalog}.{source_schema}.{source_table}"""
    )
    .filter(
        "departure_airport is not null and departure_gate is not null and economy_number_of_seats is not null and aircraftmanufacturer is not null and airline is not null and flight_number is not null"
    )
    .limit(1000)
)

df.printSchema()

In [0]:
target_table_df = spark.table(f"{target_catalog}.{target_schema}.{target_table}")
target_table_df.printSchema()
for col, dtype in target_table_df.dtypes:
    df = df.withColumn(col, df[col].cast(dtype))


In [0]:
df.write.mode("overwrite").option("mergeSchema", "true").saveAsTable(f"{target_catalog}.{target_schema}.{target_table}")